<a href="https://colab.research.google.com/github/nikunjy/learning_ml/blob/main/huggingface_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers, what can they do?

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers
!pip uninstall -y accelerate
!pip install accelerate -U

# Just using Pipelines

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
classifier("I've been waiting for a HuggingFace course my whole life.")

In [ ]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598047137260437},
 {'label': 'NEGATIVE', 'score': 0.9994558095932007}]

In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification")
classifier(
    "This is a course about the Transformers library",
    candidate_labels=["education", "politics", "business"],
)

{'sequence': 'This is a course about the Transformers library',
 'labels': ['education', 'business', 'politics'],
 'scores': [0.8445963859558105, 0.111976258456707, 0.043427448719739914]}

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

[{'generated_text': 'In this course, we will teach you how to understand and use '
                    'data flow and data interchange when handling user data. We '
                    'will be working with one or more of the most commonly used '
                    'data flows — data flows of various types, as seen by the '
                    'HTTP'}]

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

[{'generated_text': 'In this course, we will teach you how to manipulate the world and '
                    'move your mental and physical capabilities to your advantage.'},
 {'generated_text': 'In this course, we will teach you how to become an expert and '
                    'practice realtime, and with a hands on experience on both real '
                    'time and real'}]

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

[{'sequence': 'This course will teach you all about mathematical models.',
  'score': 0.19619831442832947,
  'token': 30412,
  'token_str': ' mathematical'},
 {'sequence': 'This course will teach you all about computational models.',
  'score': 0.04052725434303284,
  'token': 38163,
  'token_str': ' computational'}]

In [ ]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

[{'entity_group': 'PER', 'score': 0.99816, 'word': 'Sylvain', 'start': 11, 'end': 18}, 
 {'entity_group': 'ORG', 'score': 0.97960, 'word': 'Hugging Face', 'start': 33, 'end': 45}, 
 {'entity_group': 'LOC', 'score': 0.99321, 'word': 'Brooklyn', 'start': 49, 'end': 57}
]

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

{'score': 0.6385916471481323, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

[{'summary_text': ' America has changed dramatically during recent years . The '
                  'number of engineering graduates in the U.S. has declined in '
                  'traditional engineering disciplines such as mechanical, civil '
                  ', electrical, chemical, and aeronautical engineering . Rapidly '
                  'developing economies such as China and India, as well as other '
                  'industrial countries in Europe and Asia, continue to encourage '
                  'and advance engineering .'}]

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

[{'translation_text': 'This course is produced by Hugging Face.'}]

# AutoTokenizer and Automodel

In [6]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device : ", device)

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
inputs = tokenizer("I've been waiting for a HuggingFace course my whole life.", padding=True, truncation=True, return_tensors="pt")
inputs.to(device)
print("Input shape is 1 X number of tokens : ", inputs["input_ids"].shape)
#print(inputs["input_ids"])

# AutoModel is a general model that can pull any model from huggingface hub
# In this case we are pulling a bert which will output embeddings for each token
model = AutoModel.from_pretrained(checkpoint)
model.to(device)
outputs = model(**inputs)
print("Output last hidden state is one vector of embodding per token :", outputs.last_hidden_state.shape)

# Let's pull a model for sequence classification. So it will basically add
# a head with two outputs one for positive and one for negative. This does
# not have a hidden state anymore

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.to(device)
outputs = model(**inputs)
print("Outputs : ", outputs.logits.shape)

# last output is logits which is basically not actual values but log of actual
# values so we can softmax to convert to probabilites. Let's leave that

Device :  cuda
Input shape is 1 X number of tokens :  torch.Size([1, 16])
Output last hidden state is one vector of embodding per token : torch.Size([1, 16, 768])
Outputs :  torch.Size([1, 2])


# Tokenizers

In [60]:
# Small lesson on tokenizers
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)


sentences = [
    "yo hahah",
    "a red fox jumps over the wall and eats your apple"
]

batch = []
for sentence in sentences:
  # Instead of using tokenizer() directly let's break it down to see what all goes
  # behind it
  inputs = tokenizer.tokenize(sentence)
  print(inputs)

  # convert to numbers
  input_ids  = tokenizer.convert_tokens_to_ids(inputs)
  print(input_ids)
  batch.append(input_ids)

  # this is what goes in the model
  # notice how we convert it into a batch because that is what a model takes
  actual_input = torch.tensor([input_ids])
  print("Input IDs:", actual_input)

  # this will work on the batch of tensors
  output = model(actual_input)
  print(output.logits)
  print("=============")

# Batching, truncation and attention mask
# batch[0] is smaller than batch[1] make it equal length
original_length = len(batch[0])
batch[0] = batch[0] + [tokenizer.pad_token_id] * (len(batch[1]) - len(batch[0]))
print(batch)
batched = torch.tensor(batch)
print(model(batched).logits)

# the output for the first sentence in the batch is different this is because of
# attention mask
attention_mask = torch.ones(batched.shape[0], batched.shape[1])
attention_mask[0, original_length:] = 0
print("\nAfter attention mask")
print(model(batched, attention_mask=attention_mask).logits)


# When we pass the input to tokenizer directly it can do all the things
# like padding and truncate and attention mask
print("\nUsing tokenizer directly")
input = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
print(input['input_ids'])

# You will notice with attention mask that the padded tokens are masked with 0
print(input['attention_mask'])

['yo', 'ha', '##ha', '##h']
[10930, 5292, 3270, 2232]
Input IDs: tensor([[10930,  5292,  3270,  2232]])
tensor([[ 1.8348, -1.5315]], grad_fn=<AddmmBackward0>)
['a', 'red', 'fox', 'jumps', 'over', 'the', 'wall', 'and', 'eats', 'your', 'apple']
[1037, 2417, 4419, 14523, 2058, 1996, 2813, 1998, 20323, 2115, 6207]
Input IDs: tensor([[ 1037,  2417,  4419, 14523,  2058,  1996,  2813,  1998, 20323,  2115,
          6207]])
tensor([[-1.6322,  1.7704]], grad_fn=<AddmmBackward0>)
[[10930, 5292, 3270, 2232, 0, 0, 0, 0, 0, 0, 0], [1037, 2417, 4419, 14523, 2058, 1996, 2813, 1998, 20323, 2115, 6207]]
tensor([[ 0.6134, -0.6137],
        [-1.6322,  1.7704]], grad_fn=<AddmmBackward0>)

After attention mask
tensor([[ 1.8348, -1.5315],
        [-1.6322,  1.7704]], grad_fn=<AddmmBackward0>)

Using tokenizer directly
tensor([[  101, 10930,  5292,  3270,  2232,   102,     0,     0,     0,     0,
             0,     0,     0],
        [  101,  1037,  2417,  4419, 14523,  2058,  1996,  2813,  1998, 20323,
   

# Datsets

In [2]:
from datasets import load_dataset

from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


raw_datasets = load_dataset("glue", "mrpc")
print(raw_datasets.keys())
for k in raw_datasets['train'].features:
  print(k, " ", raw_datasets['train'].features[k])

print(raw_datasets['train'][0])

# Tokenizer does not need to work on only batch it can work on pairs as well
inputs = tokenizer("This is the first sentence.", "This is the second one.")
print(inputs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

dict_keys(['train', 'validation', 'test'])
sentence1   Value(dtype='string', id=None)
sentence2   Value(dtype='string', id=None)
label   ClassLabel(names=['not_equivalent', 'equivalent'], id=None)
idx   Value(dtype='int32', id=None)
{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}
{'input_ids': [101, 2023, 2003, 1996, 2034, 6251, 1012, 102, 2023, 2003, 1996, 2117, 2028, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

# Train a model

In [25]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification

from transformers import DataCollatorWithPadding
from transformers import Trainer
import torch
import numpy as np
import evaluate


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# We map the dataset using a function in a batched manner
# placing the dataset on GPU etc is taken care of by the dataset library
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# data_collator will just do the padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model.to(device)

training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")


def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)



with torch.no_grad():
  predictions = trainer.predict(tokenized_datasets["validation"])
  print(predictions.predictions.shape, predictions.label_ids.shape)
  preds = np.argmax(predictions.predictions, axis=-1)
  print(preds[0])
  print(predictions.label_ids[0])


  print(compute_metrics((predictions.predictions, predictions.label_ids)))

trainer.train()


with torch.no_grad():
  predictions = trainer.predict(tokenized_datasets["validation"])
  print(compute_metrics((predictions.predictions, predictions.label_ids)))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(408, 2) (408,)
0
1
{'accuracy': 0.3161764705882353, 'f1': 0.0}


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.388297,0.823529,0.867159
2,0.535400,0.380393,0.855392,0.895944
3,0.307800,0.709859,0.850490,0.896435


{'accuracy': 0.8504901960784313, 'f1': 0.896434634974533}


# Training without Trainer

In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import torch
import evaluate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

print("Number of training samples", raw_datasets["train"].num_rows)
print("Number of validation samples", raw_datasets["validation"].num_rows)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

print("Features in train:")
for k in tokenized_datasets["train"].features:
  print(k, " -> ", tokenized_datasets["train"].features[k])

tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# We have to set the format as torch here cause tokenizer would not make
# the tensors without padding and we want to apply padding using the collator
tokenized_datasets.set_format("torch")


from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
)


from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)



for batch in train_dataloader:
  break
{k: v.shape for k, v in batch.items()}
print("Sample output from the model \n", model(**batch))


from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

# Let's do 3 passes over the model using all of our dataset
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))
model.to(device)

def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


def print_eval_results():
  metric = evaluate.load("glue", "mrpc")
  model.eval()
  with torch.no_grad():
    for batch in eval_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      metric.add_batch(predictions=predictions, references=batch["labels"])
  print(metric.compute())


for epoch in range(num_epochs):
  model.train()
  for batch in train_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch)
    loss = outputs.loss
    loss.backward()

    optimizer.step()
    lr_scheduler.step()
    optimizer.zero_grad()
    progress_bar.update(1)
  print("Epoch Done: ", epoch)
  print_eval_results()

Number of training samples 3668
Number of validation samples 408
Features in train:
sentence1  ->  Value(dtype='string', id=None)
sentence2  ->  Value(dtype='string', id=None)
label  ->  ClassLabel(names=['not_equivalent', 'equivalent'], id=None)
idx  ->  Value(dtype='int32', id=None)
input_ids  ->  Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None)
token_type_ids  ->  Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)
attention_mask  ->  Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Sample output from the model 
 SequenceClassifierOutput(loss=tensor(0.8921, grad_fn=<NllLossBackward0>), logits=tensor([[-0.3731,  0.4817],
        [-0.3800,  0.4743],
        [-0.4106,  0.4493],
        [-0.3940,  0.4663],
        [-0.3860,  0.4609],
        [-0.3843,  0.4729],
        [-0.3872,  0.4934],
        [-0.3870,  0.4766]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/1377 [00:00<?, ?it/s]

Epoch Done:  0
{'accuracy': 0.8235294117647058, 'f1': 0.8741258741258742}
Epoch Done:  1
{'accuracy': 0.8529411764705882, 'f1': 0.8976109215017065}
Epoch Done:  2
{'accuracy': 0.8627450980392157, 'f1': 0.9037800687285222}


In [15]:
model.eval()

metric = evaluate.load("glue", "mrpc")
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

for batch in test_dataloader:
  batch = {k: v.to(device) for k, v in batch.items()}
  with torch.no_grad():
    model(**batch)

  outputs = model(**batch)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)
  metric.add_batch(predictions=predictions, references=batch["labels"])

print("Accruacy on the test output: \n")
print(metric.compute())



Accruacy on the test output: 

{'accuracy': 0.84, 'f1': 0.8847117794486216}
